In [1]:
import numpy as np
import torch
import torch.nn as nn
from mmcv.cnn import ConvModule, DepthwiseSeparableConvModule
from mmcv.runner import BaseModule
from mmdet.models.necks import YOLOXPAFPN
from mmdet.models.backbones import Darknet, CSPDarknet,MobileNetV2

from mmdet.models.dense_heads import YOLOXHead

from math import sqrt

from thop import profile,clever_format

CLASSES = (
        "road_cone", "car_barrier", "ground_lock", "road_pile", 
        "warning_board", "shopping_cart", "ball_stone", "sharing_bike",
        "electric_bike", "child", "others",
        )

num_cls_rearview = len(CLASSES)

img_scale = (480,640) # height, width


class NaiveModel(nn.Module):
    def __init__(self, backbone, neck, head):
        super(NaiveModel, self).__init__()
        self.backbone = backbone
        self.neck = neck
        self.head = head

    def forward(self, x):
        x = self.backbone(x)
        x = self.neck(x)
        x = self.head(x)
        return x


img_inputs = torch.randn(1, 3, img_scale[0], img_scale[1])


# default Yolox Nano Config by ti



In [2]:
test_backbone = CSPDarknet(
    deepen_factor=0.33,
    widen_factor=0.25,
    use_depthwise=True)
test_neck = YOLOXPAFPN(
    in_channels=[64,128,256],
    out_channels=64,
    num_csp_blocks=1,
    use_depthwise=True,
   )
head = YOLOXHead(
     num_classes=num_cls_rearview, 
        in_channels=64, 
        feat_channels=64, 
        use_depthwise=True,
)
m_ori = NaiveModel(
        test_backbone,
        test_neck,
        head
)
outputs_ori = m_ori(img_inputs)
print(outputs_ori[0][0].shape)

ops, params = profile(m_ori, inputs=(img_inputs,))
print('model  #flops: {},\t#params: {}'.format(*clever_format([ops, params], "%.3f")))


torch.Size([1, 11, 60, 80])
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
model  #flops: 959.770M,	#params: 898.704K


# customized yolox lite Config by ti


In [12]:
use_depthwise = False
test_backbone = CSPDarknet(
    arch='P4',
    deepen_factor=0.5,
    widen_factor=0.5,
    out_indices=(3,),
    use_depthwise=use_depthwise,
    spp_kernal_sizes=(5, 9, 13),
    act_cfg=dict(type='LeakyReLU', negative_slope=0.1))
test_neck = YOLOXPAFPN(
    in_channels=[256,],
    out_channels=256,
    num_csp_blocks=2,
    use_depthwise=use_depthwise,
    upsample_cfg=dict(scale_factor=2, mode='bilinear'),
    conv_cfg=None,
    norm_cfg=dict(type='BN', momentum=0.03, eps=0.001),
    act_cfg=dict(type='LeakyReLU', negative_slope=0.1),
   )
head = YOLOXHead(
    num_classes=num_cls_rearview,
    in_channels=256,
    feat_channels=128,
    strides=[16],
    use_depthwise=use_depthwise,
    act_cfg=dict(type='LeakyReLU', negative_slope=0.1),
)
img_inputs = torch.randn(1, 3, img_scale[0], img_scale[1])


m = NaiveModel(
        test_backbone,
        test_neck,
        head
)
outputs = m(img_inputs)
print(outputs[0][0].shape)
# torch.onnx.export(m, img_inputs, "litedbg.onnx", verbose=True, opset_version=11)

ops, params = profile(m, inputs=(img_inputs,))
print('model  #flops: {},\t#params: {}'.format(*clever_format([ops, params], "%.3f")))

torch.Size([1, 11, 30, 40])
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register count_relu() for <class 'torch.nn.modules.activation.LeakyReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
model  #flops: 5.170G,	#params: 2.527M
